In [1]:
import numpy as np
from ShillingAttack import Attack
import random 
import math
import csv
from ML100KPrePreocessing import ML100KRatingMatrix
from AmazonSoftwarePreprocessing import AmazonSoftwareRatingMatrix
from AmazonFashionPreProcessing import AmazonFashionRatingMatrix

In [2]:
class RandomAttack(Attack):
    def __init__(self,attacksize,fillersize,targetItems,k,dataset):
        super().__init__(attacksize,fillersize,targetItems,k,dataset)
    
    def createProfiles(self):
        
        self.pushItem() #use function to nuke items in case of a nuke attack 
        self.fillerItems = self.randomNumberGenerator()
        
        nonZeroRatingMatrix = self.ratingMatrix[self.ratingMatrix != 0]
        
        mean = np.mean(nonZeroRatingMatrix)
        sd = np.std(nonZeroRatingMatrix)
        
        max = mean + 3*sd
        min = mean - 3*sd
        
        #print('The mean of the dataset is: {}'.format(mean))  #Used for testing purposes
        #print('The sd of the dataset is: {}'.format(sd)) #Used for testing purposes
        
        for i in range(self.attackSize):
            j = 0
            while j < self.fillerSize:
                item = random.choice(self.fillerItems)
                if item not in self.shillingProfiles[i]:
                    rating = random.gauss(mean,sd)
                    
                    rating = math.ceil(1 + ((rating - min)/(max - min)) * 4) if max - min != 0 else 1
                    #normalizing the output of the gaussian to be between 1 and 5
                    
                    if rating > 5:
                        rating = 5
                    if rating < 1:
                        rating = 1
                        
                    self.shillingProfiles[i][item-1] = rating
                    j += 1
                    
    def mountAttack(self):
        self.createProfiles()
        self.injectProfiles()
        
        if isinstance(self.RM,ML100KRatingMatrix): 
        
            with open('ML100K_RandomAttack.csv','w') as file:
                docWriter = csv.writer(file, delimiter = ',')
                docWriter.writerows(self.ratingMatrix)
                
        elif isinstance(self.RM,AmazonFashionRatingMatrix):
        
            with open('AmazonFashion_RandomAttack.csv','w') as file:
                docWriter = csv.writer(file, delimiter = ',')
                docWriter.writerows(self.ratingMatrix)
        else: 
            with open('AmazonSoftware_RandomAttack.csv','w') as file:
                docWriter = csv.writer(file, delimiter = ',')
                docWriter.writerows(self.ratingMatrix)

In [3]:
def test ():
    Test = RandomAttack(95,169,[0,1,2],0,0)#change last input to change dataset
    Test.mountAttack()

In [4]:
test()